In [1]:
import warnings
warnings.filterwarnings('ignore')

## 1. One-Hot Encoding

In [2]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

word_dict = {
    'school' : np.array([[1, 0, 0]]),
    'study'  : np.array([[0, 1, 0]]),
    'workout': np.array([[0, 0, 1]])
}

cosine_school_study   = cosine_similarity(word_dict['school'], word_dict['study'])
cosine_school_workout = cosine_similarity(word_dict['school'], word_dict['workout'])

print(f'cosine-similarity between schooland study  : {cosine_school_study}')
print(f'cosine-similarity between schooland workout: {cosine_school_workout}')

cosine-similarity between schooland study  : [[0.]]
cosine-similarity between schooland workout: [[0.]]


## 2. Bag of Words

In [3]:
documents = [
    "I love AI and AI loves me",
    "AI is amazing and I love it",
    "AI is changing the world",
    "I love learning AI",
    "AI makes life easier"
]

# 단어 사전 만들기
word_set = set()
for doc in documents:
    words = doc.lower().split()  # 소문자로 변환 후 단어 분리
    word_set.update(words)

# 정렬하여 인덱싱 고정
word_list = sorted(word_set)
print("✅ 단어 리스트:", word_list)

# 문장별 단어 등장 횟수 계산(BoW 행렬 생성)
bow_matrix = []
for doc in documents:
    words = doc.lower().split()
    word_count = [words.count(word) for word in word_list]
    bow_matrix.append(word_count)

print("✅ BoW 행렬:")
for row in bow_matrix:
    print(row)

✅ 단어 리스트: ['ai', 'amazing', 'and', 'changing', 'easier', 'i', 'is', 'it', 'learning', 'life', 'love', 'loves', 'makes', 'me', 'the', 'world']
✅ BoW 행렬:
[2, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0]
[1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1]
[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]


## 3. Word2Vec: CBOW

In [4]:
import numpy as np
import random

# 1. 샘플 문장 데이터 (단순한 문장 몇 개)
corpus = [
  "AI is changing the world",
  "I love learning AI",
  "AI is powerful and useful",
  "Deep learning is a part of AI",
  "Natural language processing is exciting"
]

# 2. 데이터 전처리: 단어 토큰화
tokenized_corpus = [sentence.lower().split() for sentence in corpus]

# 3. 단어 사전 구축
word_list = sorted(set(sum(tokenized_corpus, [])))  # 중복 없는 단어 리스트
word2idx = {word: i for i, word in enumerate(word_list)}  # 단어 -> 인덱스
idx2word = {i: word for word, i in word2idx.items()}  # 인덱스 -> 단어
vocab_size = len(word_list)  # 단어 개수

print("\n✅ 단어 사전:", word2idx)


✅ 단어 사전: {'a': 0, 'ai': 1, 'and': 2, 'changing': 3, 'deep': 4, 'exciting': 5, 'i': 6, 'is': 7, 'language': 8, 'learning': 9, 'love': 10, 'natural': 11, 'of': 12, 'part': 13, 'powerful': 14, 'processing': 15, 'the': 16, 'useful': 17, 'world': 18}


In [5]:
# 4. 학습 데이터 생성 (CBOW 방식)
window_size = 2  # 중심 단어 기준 좌우 몇 개의 단어를 고려할지
training_data = []
for sentence in tokenized_corpus:
  for i, word in enumerate(sentence):
    context_words = []

    # 주변 단어 찾기 (윈도우 크기 고려)
    for j in range(-window_size, window_size + 1):
      if j == 0 or i + j < 0 or i + j >= len(sentence):
        continue
      context_words.append(word2idx[sentence[i + j]])

    if len(context_words) > 0:
      training_data.append((context_words, word2idx[word]))  # (주변 단어들, 중심 단어)

print("\n✅ 학습 데이터 (주변 단어 → 중심 단어):")
print([([idx2word[c] for c in ctx], idx2word[t]) for ctx, t in training_data])


✅ 학습 데이터 (주변 단어 → 중심 단어):
[(['is', 'changing'], 'ai'), (['ai', 'changing', 'the'], 'is'), (['ai', 'is', 'the', 'world'], 'changing'), (['is', 'changing', 'world'], 'the'), (['changing', 'the'], 'world'), (['love', 'learning'], 'i'), (['i', 'learning', 'ai'], 'love'), (['i', 'love', 'ai'], 'learning'), (['love', 'learning'], 'ai'), (['is', 'powerful'], 'ai'), (['ai', 'powerful', 'and'], 'is'), (['ai', 'is', 'and', 'useful'], 'powerful'), (['is', 'powerful', 'useful'], 'and'), (['powerful', 'and'], 'useful'), (['learning', 'is'], 'deep'), (['deep', 'is', 'a'], 'learning'), (['deep', 'learning', 'a', 'part'], 'is'), (['learning', 'is', 'part', 'of'], 'a'), (['is', 'a', 'of', 'ai'], 'part'), (['a', 'part', 'ai'], 'of'), (['part', 'of'], 'ai'), (['language', 'processing'], 'natural'), (['natural', 'processing', 'is'], 'language'), (['natural', 'language', 'is', 'exciting'], 'processing'), (['language', 'processing', 'exciting'], 'is'), (['processing', 'is'], 'exciting')]


In [6]:
# 5. 신경망 학습을 위한 파라미터 설정
embedding_dim = 10  # 단어 벡터 차원 수
learning_rate = 0.005  # 수정된 학습률 (조금 증가)
epochs = 1000  # 학습 반복 횟수

# 6. 가중치 초기화 (Xavier 초기화 적용)
W1 = np.random.randn(vocab_size, embedding_dim) / np.sqrt(vocab_size)
W2 = np.random.randn(embedding_dim, vocab_size) / np.sqrt(embedding_dim)

# 7. 원-핫 벡터 변환 함수
def one_hot_encoding(word_index, vocab_size):
  one_hot_vector = np.zeros(vocab_size)
  one_hot_vector[word_index] = 1
  return one_hot_vector

In [7]:
# 8. CBOW 학습 과정
for epoch in range(epochs):
  random.shuffle(training_data)  # 데이터 셔플링 추가
  loss = 0
  for context_words, target in training_data:
    # 평균 컨텍스트 벡터 계산
    context_vectors = [one_hot_encoding(w, vocab_size) for w in context_words]
    x = np.mean(context_vectors, axis=0)  # 여러 개의 원-핫 벡터 평균

    # 순전파 (Forward propagation)
    hidden_layer = np.dot(x, W1)
    output_layer = np.dot(hidden_layer, W2)

    # Softmax 오버플로우 방지
    exp_out = np.exp(output_layer - np.max(output_layer))
    y_pred = exp_out / np.sum(exp_out)

    # 정답 벡터
    y_true = one_hot_encoding(target, vocab_size)

    # 손실 계산 (Cross-Entropy Loss, 정규화 추가)
    loss += -np.sum(y_true * np.log(y_pred + 1e-9)) / len(training_data)

    # 역전파 (Backpropagation)
    error = y_pred - y_true
    W2 -= learning_rate * np.outer(hidden_layer, error)
    W1 -= learning_rate * np.outer(x, np.dot(error, W2.T))

  # 100번마다 손실 출력
  if (epoch + 1) % 100 == 0:
    print(f"Epoch {epoch+1}/{epochs} - Avg Loss: {loss:.6f}")

Epoch 100/1000 - Avg Loss: 2.650060
Epoch 200/1000 - Avg Loss: 2.243334
Epoch 300/1000 - Avg Loss: 1.791765
Epoch 400/1000 - Avg Loss: 1.405406
Epoch 500/1000 - Avg Loss: 1.091637
Epoch 600/1000 - Avg Loss: 0.849857
Epoch 700/1000 - Avg Loss: 0.662646
Epoch 800/1000 - Avg Loss: 0.520169
Epoch 900/1000 - Avg Loss: 0.415750
Epoch 1000/1000 - Avg Loss: 0.340371


In [8]:
# 9. 학습된 단어 벡터 출력
print("\n✅ 학습된 'AI'의 벡터:")
print(W1[word2idx["ai"]])

# 10. 단어 유사도 계산 함수
def cosine_similarity(vec1, vec2):
  return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

print("\n✅ 'AI'와 'learning'의 유사도:")
print(cosine_similarity(W1[word2idx["ai"]], W1[word2idx["learning"]]))


✅ 학습된 'AI'의 벡터:
[ 2.07911962 -0.95475846  0.38795507  1.45212952 -0.2713259   2.6087246
  0.06722047  0.36669713 -1.67076698 -0.67336714]

✅ 'AI'와 'learning'의 유사도:
0.2374158598807644


## 4. Word2Vec: Skip-Gram

In [9]:
import numpy as np
import random

# 1. 샘플 문장 데이터 (단순한 문장 몇 개)
corpus = [
  "AI is changing the world",
  "I love learning AI",
  "AI is powerful and useful",
  "Deep learning is a part of AI",
  "Natural language processing is exciting"
]

# 2. 데이터 전처리: 단어 토큰화
tokenized_corpus = [sentence.lower().split() for sentence in corpus]

# 3. 단어 사전 구축
word_list = sorted(set(sum(tokenized_corpus, [])))  # 중복 없는 단어 리스트
word2idx = {word: i for i, word in enumerate(word_list)}  # 단어 -> 인덱스
idx2word = {i: word for word, i in word2idx.items()}  # 인덱스 -> 단어
vocab_size = len(word_list)  # 단어 개수

print("\n✅ 단어 사전:", word2idx)


✅ 단어 사전: {'a': 0, 'ai': 1, 'and': 2, 'changing': 3, 'deep': 4, 'exciting': 5, 'i': 6, 'is': 7, 'language': 8, 'learning': 9, 'love': 10, 'natural': 11, 'of': 12, 'part': 13, 'powerful': 14, 'processing': 15, 'the': 16, 'useful': 17, 'world': 18}


In [10]:
# 4. 학습 데이터 생성 (Skip-gram 방식)
window_size = 2  # 중심 단어 기준 좌우 몇 개의 단어를 고려할지
training_data = []
for sentence in tokenized_corpus:
  for i, word in enumerate(sentence):
    center_word = word2idx[word]
    context_words = []

    # 주변 단어 찾기 (윈도우 크기 고려)
    for j in range(-window_size, window_size + 1):
      if j == 0 or i + j < 0 or i + j >= len(sentence):
        continue
      context_words.append(word2idx[sentence[i + j]])

    # 학습 데이터 추가
    for context in context_words:
      training_data.append((center_word, context))

print("\n✅ 학습 데이터 (중심 단어 → 주변 단어):")
print([(idx2word[c], idx2word[t]) for c, t in training_data])


✅ 학습 데이터 (중심 단어 → 주변 단어):
[('ai', 'is'), ('ai', 'changing'), ('is', 'ai'), ('is', 'changing'), ('is', 'the'), ('changing', 'ai'), ('changing', 'is'), ('changing', 'the'), ('changing', 'world'), ('the', 'is'), ('the', 'changing'), ('the', 'world'), ('world', 'changing'), ('world', 'the'), ('i', 'love'), ('i', 'learning'), ('love', 'i'), ('love', 'learning'), ('love', 'ai'), ('learning', 'i'), ('learning', 'love'), ('learning', 'ai'), ('ai', 'love'), ('ai', 'learning'), ('ai', 'is'), ('ai', 'powerful'), ('is', 'ai'), ('is', 'powerful'), ('is', 'and'), ('powerful', 'ai'), ('powerful', 'is'), ('powerful', 'and'), ('powerful', 'useful'), ('and', 'is'), ('and', 'powerful'), ('and', 'useful'), ('useful', 'powerful'), ('useful', 'and'), ('deep', 'learning'), ('deep', 'is'), ('learning', 'deep'), ('learning', 'is'), ('learning', 'a'), ('is', 'deep'), ('is', 'learning'), ('is', 'a'), ('is', 'part'), ('a', 'learning'), ('a', 'is'), ('a', 'part'), ('a', 'of'), ('part', 'is'), ('part', 'a'), ('par

In [11]:
# 5. 신경망 학습을 위한 파라미터 설정
embedding_dim = 10  # 단어 벡터 차원 수
learning_rate = 0.005  # 수정된 학습률 (조금 증가)
epochs = 1000  # 학습 반복 횟수

# 6. 가중치 초기화 (Xavier 초기화 적용)
W1 = np.random.randn(vocab_size, embedding_dim) / np.sqrt(vocab_size)
W2 = np.random.randn(embedding_dim, vocab_size) / np.sqrt(embedding_dim)

# 7. 원-핫 벡터 변환 함수
def one_hot_encoding(word_index, vocab_size):
  one_hot_vector = np.zeros(vocab_size)
  one_hot_vector[word_index] = 1
  return one_hot_vector

In [12]:
# 8. Skip-gram 학습 과정
for epoch in range(epochs):
  random.shuffle(training_data)  # 데이터 셔플링 추가
  loss = 0
  for center, context in training_data:
    # 원-핫 인코딩 (입력)
    x = one_hot_encoding(center, vocab_size)

    # 순전파 (Forward propagation)
    hidden_layer = np.dot(x, W1)
    output_layer = np.dot(hidden_layer, W2)

    # Softmax 오버플로우 방지
    exp_out = np.exp(output_layer - np.max(output_layer))
    y_pred = exp_out / np.sum(exp_out)

    # 정답 벡터
    y_true = one_hot_encoding(context, vocab_size)

    # 손실 계산 (Cross-Entropy Loss, 정규화 추가)
    loss += -np.sum(y_true * np.log(y_pred + 1e-9)) / len(training_data)

    # 역전파 (Backpropagation)
    error = y_pred - y_true
    W2 -= learning_rate * np.outer(hidden_layer, error)
    W1 -= learning_rate * np.outer(x, np.dot(error, W2.T))

  # 100번마다 손실 출력
  if (epoch + 1) % 100 == 0:
    print(f"Epoch {epoch+1}/{epochs} - Avg Loss: {loss:.6f}")

Epoch 100/1000 - Avg Loss: 2.373285
Epoch 200/1000 - Avg Loss: 1.870219
Epoch 300/1000 - Avg Loss: 1.654505
Epoch 400/1000 - Avg Loss: 1.587934
Epoch 500/1000 - Avg Loss: 1.563087
Epoch 600/1000 - Avg Loss: 1.551839
Epoch 700/1000 - Avg Loss: 1.545732
Epoch 800/1000 - Avg Loss: 1.542455
Epoch 900/1000 - Avg Loss: 1.540409
Epoch 1000/1000 - Avg Loss: 1.538818


In [13]:
# 9. 학습된 단어 벡터 출력
print("\n✅ 학습된 'AI'의 벡터:")
print(W1[word2idx["ai"]])

# 10. 단어 유사도 계산 함수
def cosine_similarity(vec1, vec2):
  return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

print("\n✅ 'AI'와 'learning'의 유사도:")
print(cosine_similarity(W1[word2idx["ai"]], W1[word2idx["learning"]]))


✅ 학습된 'AI'의 벡터:
[-0.62533287  0.78715706  0.82410121 -0.88558919 -0.65744717  0.40566861
 -0.18185193  1.91765001  0.18919656 -1.66323656]

✅ 'AI'와 'learning'의 유사도:
-0.17754438125541705
